<a href="https://colab.research.google.com/github/hluling/ph-dash-dev/blob/master/ph_dash_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook setup and Github setup
2022/04/22



In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd /content/drive/MyDrive/Github
# !git init ph-dash-dev
# %cd ph-dash-dev
# !git remote add origin https://github.com/hluling/ph-dash-dev
#!git add .
# !git commit -m 'repo setup'
# !git config --global user.email "hluling@outlook.com"
# !git push -u origin master

# Install Libraries

In [1]:
import datetime
import requests
import pandas as pd
from io import StringIO
from datetime import date

## Query API

In [ ]:
response = requests.get("https://api.gdeltproject.org/api/v2/tv/tv?query=%22puerto%20rico%22%20(station:CNN%20OR%20station:FOXNEWS)&mode=timelinevol&format=html&datanorm=perc&timelinesmooth=5&datacomb=sep&last24=yes&timezoom=no&STARTDATETIME=20170829120000&ENDDATETIME=20171007120000")

In [ ]:
response_csv = requests.get("https://api.gdeltproject.org/api/v2/tv/tv?query=%22puerto%20rico%22%20(station:CNN%20OR%20station:FOXNEWS)&mode=timelinevol&format=html&datanorm=perc&timelinesmooth=5&format=csv&datacomb=sep&last24=yes&timezoom=no&STARTDATETIME=20210829120000&ENDDATETIME=20211007120000")
text=StringIO(response_csv.content.decode('utf-8'))

In [ ]:
df=pd.read_csv(text)

In [ ]:
df

,Date (Daily +00:00: 08/29/2021 - 10/07/2021),Series,Value
0,2021-09-02,CNN,0.0170
1,2021-09-03,CNN,0.0170
2,2021-09-04,CNN,0.0170
3,2021-09-05,CNN,0.0042
4,2021-09-06,CNN,0.0042
...,...,...,...
67,2021-10-03,FOXNEWS,0.0241
68,2021-10-04,FOXNEWS,0.0241
69,2021-10-05,FOXNEWS,0.0241
70,2021-10-06,FOXNEWS,0.0098


## Search API: a quick test

In [ ]:
query_url = "https://api.gdeltproject.org/api/v2/tv/tv?query=(kremlin%20OR%20russia%20OR%20putin)%20(ukraine%20OR%20ukrainian%20OR%20zelenskyy)%20market:%22National%22&mode=timelinevol&format=html&datanorm=perc&format=csv&timelinesmooth=5&datacomb=sep&timezoom=yes&STARTDATETIME=20220224120000&ENDDATETIME=20220630120000"

In [9]:
def to_df(queryurl):
  response = requests.get(queryurl)
  content_text = StringIO(response.content.decode('utf-8'))
  df = pd.read_csv(content_text)
  return df

In [ ]:
test_df = to_df(query_url)

In [ ]:
test_df # each day corresponds to a group of channels
# value is the percentage of number of matched 15-seconds clips over the number of monitored 15-seconds clips (because stations may devote different lengths of time to tv news)
# if want raw count, then set datanorm=raw (now datanorm=perc)

,Date (Daily +00:00: 02/24/2022 - 06/30/2022),Series,Value
0,2022-02-28,BLOOMBERG,4.6970
1,2022-03-01,BLOOMBERG,4.3581
2,2022-03-02,BLOOMBERG,3.7279
3,2022-03-03,BLOOMBERG,4.4902
4,2022-03-04,BLOOMBERG,4.5878
...,...,...,...
1102,2022-06-26,MSNBC,0.1914
1103,2022-06-27,MSNBC,0.2816
1104,2022-06-28,MSNBC,0.3406
1105,2022-06-29,MSNBC,0.4955


In [ ]:
test_df.head(30)

In [ ]:
test_df.Series.unique()

array(['BLOOMBERG', 'CNBC', 'CNN', 'CSPAN', 'CSPAN2', 'CSPAN3', 'FBC',
       'FOXNEWS', 'MSNBC'], dtype=object)

In [ ]:
# The meat of the code for the article starts here

## Retrieve and prepare data for dashboard
2022/7/7

In [2]:
today = date.today()
today_str = today.strftime("%Y%m%d")
print(today_str)

20221129


In [3]:
start_day = today - datetime.timedelta(60)

In [4]:
start_day_str = start_day.strftime("%Y%m%d")

In [5]:
print(start_day_str)

20220930


In [6]:
query_url_ukr = f"https://api.gdeltproject.org/api/v2/tv/tv?query=(ukraine%20OR%20ukrainian%20OR%20zelenskyy%20OR%20kiev)%20market:%22National%22&mode=timelinevol&format=html&datanorm=perc&format=csv&timelinesmooth=5&datacomb=sep&timezoom=yes&STARTDATETIME={start_day_str}120000&ENDDATETIME={today_str}120000"

In [7]:
query_url_rus = f"https://api.gdeltproject.org/api/v2/tv/tv?query=(kremlin%20OR%20russia%20OR%20putin%20OR%20moscow%20OR%20russian)%20market:%22National%22&mode=timelinevol&format=html&datanorm=perc&format=csv&timelinesmooth=5&datacomb=sep&timezoom=yes&STARTDATETIME={start_day_str}120000&ENDDATETIME={today_str}120000"

In [10]:
df_ukr = to_df(query_url_ukr)

In [11]:
df_rus = to_df(query_url_rus)

In [12]:
df_ukr.head()

,Date (Daily +00:00: 09/30/2022 - 11/27/2022),Series,Value
0,2022-10-04,BLOOMBERG,1.5418
1,2022-10-05,BLOOMBERG,1.4015
2,2022-10-06,BLOOMBERG,1.6101
3,2022-10-07,BLOOMBERG,1.6264
4,2022-10-08,BLOOMBERG,1.1735


In [13]:
df_rus.head()

,Date (Daily +00:00: 09/30/2022 - 11/27/2022),Series,Value
0,2022-10-04,BLOOMBERG,2.0916
1,2022-10-05,BLOOMBERG,2.1513
2,2022-10-06,BLOOMBERG,2.8406
3,2022-10-07,BLOOMBERG,3.1954
4,2022-10-08,BLOOMBERG,2.4405


In [14]:
df_ukr = df_ukr.rename(columns={df_ukr.columns[0]: "date_col"})
df_rus = df_rus.rename(columns={df_rus.columns[0]: "date_col"})

In [13]:
df_ukr.dtypes

date_col     object
Series       object
Value       float64
dtype: object

In [15]:
df_ukr['date_col'] = pd.to_datetime(df_ukr['date_col'])

In [15]:
df_rus.dtypes

date_col     object
Series       object
Value       float64
dtype: object

In [16]:
df_rus['date_col'] = pd.to_datetime(df_rus['date_col'])

In [17]:
df_rus = df_rus[[x in ['CNN', 'FOXNEWS', 'MSNBC'] for x in df_rus.Series]]

In [18]:
df_ukr = df_ukr[[x in ['CNN', 'FOXNEWS', 'MSNBC'] for x in df_ukr.Series]]

# Dashboard part starts here

In [19]:
!pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 11.6 MB/s 
     |████████████████████████████████| 1.6 MB 43.0 MB/s 


In [20]:
!pip install dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
!pip install dash_bootstrap_components

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 5.4 MB/s 


In [22]:
import dash
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px

In [23]:
# Initiate a dashboard instance
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LITERA])
server = app.server

# Dashboard front end
app.layout = dbc.Container(
    [   dbc.Row([
        dbc.Col([html.H1('US National Television News Coverage of the War in Ukraine')],
        #html.Hr()],
        className="text-center mt-3 mb-1")
    ]
    ),
        dbc.Row([
            dbc.Label("Select a date range:", className="fw-bold")
    ]),
    
     dbc.Row([
              dcc.DatePickerRange(
                id='date-range',
                min_date_allowed=df_ukr['date_col'].min().date(),
                max_date_allowed=df_ukr['date_col'].max().date(),
                initial_visible_month=df_ukr['date_col'].min().date(),
                start_date=df_ukr['date_col'].min().date(),
                end_date=df_ukr['date_col'].max().date()
              )
    ]),

     dbc.Row([
              dbc.Col(dcc.Graph(id='line-graph-ukr'), 
                      )
     ]),

    dbc.Row([
              dbc.Col(dcc.Graph(id='line-graph-rus'), 
                      )
     ])

    ])

# Dashboard back end
@app.callback(
    Output('line-graph-ukr', 'figure'),
    Output('line-graph-rus', 'figure'),
    Input('date-range', 'start_date'),
    Input('date-range', 'end_date')   
)
def update_output(start_date, end_date):
    mask_ukr = (df_ukr['date_col'] >= start_date) & (df_ukr['date_col'] <= end_date)
    mask_rus = (df_rus['date_col'] >= start_date) & (df_rus['date_col'] <= end_date)

    df_ukr_filtered = df_ukr.loc[mask_ukr]
    df_rus_filtered = df_rus.loc[mask_rus]
    
    line_fig_ukr = px.line(df_ukr_filtered, x="date_col", y="Value", 
                     color='Series', title="Coverage of Ukranian Keywords")
    line_fig_rus = px.line(df_rus_filtered, x='date_col', y='Value', 
                     color='Series', title="Coverage of Russian Keywords")

    line_fig_ukr.update_layout(
                   xaxis_title='Date',
                   yaxis_title='Percentage of Airtime')
    
    line_fig_rus.update_layout(
                   xaxis_title='Date',
                   yaxis_title='Percentage of Airtime')

    line_fig_ukr.update_xaxes(tickformat="%b %d<br>%Y")
    line_fig_rus.update_xaxes(tickformat="%b %d<br>%Y")
    
    return line_fig_ukr, line_fig_rus

In [24]:
# Show dashboard inside the notebook
app.run_server(debug=True, mode="inline")

<IPython.core.display.Javascript object>

In [ ]:
# Show dashboard in browser (in a separate window)
app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
!python --version

Python 3.7.13


# In lesson form (rough)
11/28/2022

# Title: Creat a Dashboard for Interactive Data Visualization with Dash in Python

## Contents
  * [Introduction](#introduction)
  * [Lesson Goals](#lesson-goals)
  * [A Research Question for Demonstration](#research-question)
  * [Dataset](#dataset)
  * [Prepare for the Lesson](#prepare-for-the-lesson)
    * Prerequisites
    * Create a Virtual Environment
    * Install Libraries
  * [An Idea of a Simple Dashboard](#dashboard-idea)
  * [Coding the Dashboard](#coding-dashboard)
  * [Deploying the Dashboard](#deploying-dashboard)
  * [Conclusion](#conclusion)



## Introduction

To advance open scholarship in humanities, it is important to make the research output more accessible to other scholars and the general public. It would be beneficial for scholars to explore ways to better engage with a broader audience. Creating a web-based interactive dashboard to visualize data has become a popular method nowadays to achieve this goal. The current lesson will walk you through the process of creating an interactive dashboard using the latest data from a publicly available dataset using the Dash library in Python. For demonstration, this lesson uses a research question in the field of media studies: How do the U.S. television stations cover the war in Ukraine. The dataset used in this lesson is a publicly available data of transcription texts of television news.

This lesson contributes to the existing Programming Historian lessons by adding a tutorial focused on creating an interactive web-based dashboard in Python ([see a similar English lesson focused on using Shiny in R](https://programminghistorian.org/en/lessons/shiny-leaflet-newspaper-map-tutorial)).

## Lesson Goals
You will learn how to:
  * Retrieve latest data using [Application Programming Interface (API)](https://en.wikipedia.org/wiki/API)
  * Create the frontend of dashboard that determines how it looks
  * Create the backend of dashboard that determines how users interact with it
  * Deploy dashboard onto the web using a free service

Other essential steps such as installing necessary libraries, setting up a [virtual environment](https://docs.python.org/3/library/venv.html#venv-def), and manipulating the downloaded data will be included when appropriate as well.

## A Research Question for Demonstration
The research question in this lesson is whether the U.S. television stations have covered the war in Ukraine in a balance way. One way to address this research question is to compare whether the stations have mentioned the keywords related to Ukraine as frequently as the keywords related to Russia. Further, we want to compare the coverage frequency among some major stations.  

## Dataset
To answer the research question, this lesson uses a free and open dataset from the Internet Archive's [Television Explorer](https://blog.archive.org/2016/12/20/new-research-tool-for-visualizing-two-million-hours-of-television-news/). This dataset tracks how much airtime of news coverage television stations include certain keywords at the resolution of 15 seconds. The keyword searches are based on the text of closed captioning. The tool that will be used to retrieve the data is the [2.0 TV API](https://blog.gdeltproject.org/gdelt-2-0-television-api-debuts/) made available by the Global Database of Events, Language and Tone (GDELT).

## Prepare for the Lesson

### Prerequisites
  * Python 3 (3.7.13 or later). See [Mac Installation](https://programminghistorian.org/lessons/mac-installation), [Windows Installation](https://programminghistorian.org/lessons/windows-installation), or [Linux Installation](https://programminghistorian.org/lessons/linux-installation)
  * Command line. For introductions, see [Windows here](https://programminghistorian.org/en/lessons/intro-to-powershell) and [macOS/Linux here](https://programminghistorian.org/en/lessons/intro-to-bash)
  * A text editor (e.g., Atom, Visual Studio Code) to write Python code
  * A web browser

Optional: Jupyter Notebook. If you prefer to run the code example in Jupyter Notebook, you'll need to install it (described later).

### Create a Virtual Environment
To avoid conflicts in library versions among multiple Python projects, it is a common good practice to create a virtual environment for each project. We will do so for this lesson.

There are several ways to create a virtual environment. One way is to use `conda` ([see this lesson for more details](https://programminghistorian.org/en/lessons/visualizing-with-bokeh#prerequisites)). This is a good option if you are already using [Anaconda](https://docs.conda.io/projects/conda/en/latest/glossary.html?highlight=anaconda#anaconda) for more data-science-oriented projects. Assuming that you are starting fresh, it would be more appropriate to go for a more lightweight method by using [virtualenv](https://virtualenv.pypa.io/en/latest/). To install, open a command line window and run `pip install virtualenv`.

Next, create a folder at your preferred location for the current lesson and name it *ph-dash*. In your command line, navigate to the *ph-dash* directory. To create a virtual environment called *venv*, run `virtualenv venv`. Then, you need to activate the virtual environment by running:

    venv\Scripts\activate # For Windows
    source venv/bin/activate # For macOS/Linux

If properly executed, you will see a pair of parentheses around *venv*, the name of the created virtual environment, at the start of the current line in your command line window. Now, you are in an isolated development environment with a specific version of Python and a specific list of libraries with their specific versions. To demonstrate the idea, within the virtual environment just activated, if you run `pip list` to get a list of the libraries currently installed under the *venv* environment, you will not see the virtualenv library. The reason is that we installed this library a moment ago system-wide, but now we are in an isolated virtual environment. When you are done writing code, to deactivate, just run `deactivate`.

### Install Libraries
Once our virtual environment is set up, we are ready to install several third-party libraries needed for the current lesson. With the virtual environment still in the activated mode, run `pip install requests pandas dash dash_bootstrap_components`.

## An Idea of a Simple Dashboard
To address the research question, we can envision a dashboard where there are two line graphs, one showing the trend of Russia-related terms and the other for the trend of Ukriane-related terms mentioned by television networks. More specifically, in either of the line graph, the y-axis represents the relative frequency of key terms mentioned by a certain national station, and the x-axis represents dates. In addition, there are multiple lines, each representing one station. A basic interactive component is a date-range selector that users can use to specify a range of dates, and the line graphs will be updated upon selection.


## Coding the Dashboard

### Import Libraries

In [25]:
import datetime
import requests
import pandas as pd
from io import StringIO
from datetime import date
import dash
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px

### Retrieve Data Using API

In [26]:
today = date.today()
today_str = today.strftime("%Y%m%d")
start_day = today - datetime.timedelta(60)
start_day_str = start_day.strftime("%Y%m%d")

Code explanation: We first define a range of dates for the complete dataset we want to retrieve using the API. The goal here is to create two string objects: `today_str` and `start_day_str`. Note that here we restrict the range to be 60 days for demonstration purpose only.



In [28]:
query_url_ukr = f"https://api.gdeltproject.org/api/v2/tv/tv?query=(ukraine%20OR%20ukrainian%20OR%20zelenskyy%20OR%20zelensky%20OR%20kiev)%20market:%22National%22&mode=timelinevol&format=html&datanorm=perc&format=csv&timelinesmooth=5&datacomb=sep&timezoom=yes&STARTDATETIME={start_day_str}120000&ENDDATETIME={today_str}120000"

In [29]:
query_url_rus = f"https://api.gdeltproject.org/api/v2/tv/tv?query=(kremlin%20OR%20russia%20OR%20putin%20OR%20moscow%20OR%20russian)%20market:%22National%22&mode=timelinevol&format=html&datanorm=perc&format=csv&timelinesmooth=5&datacomb=sep&timezoom=yes&STARTDATETIME={start_day_str}120000&ENDDATETIME={today_str}120000"

Code explanation: We create two string objects for query: one for Ukraine-related terms and one for Russia-related terms. The parameters to be specified include keywords, geographic market, output mode, output format, range of dates, etc. See [this documentation](https://blog.gdeltproject.org/gdelt-2-0-television-api-debuts/) for a complete description of query parameters. 

In [ ]:
def to_df(queryurl):
  response = requests.get(queryurl)
  content_text = StringIO(response.content.decode('utf-8'))
  df = pd.read_csv(content_text)
  return df

Code explanation: Now we use the `requests` library to execute the queries and transform the query results into a `pandas` dataframe. To do this, we create a function called `to_df()` to streamline the workflow.

In [31]:
df_ukr = to_df(query_url_ukr)

In [32]:
df_rus = to_df(query_url_rus)

In [33]:
# Take a look at the retrieved dataframe
df_ukr.head()

,Date (Daily +00:00: 09/30/2022 - 11/27/2022),Series,Value
0,2022-10-04,BLOOMBERG,1.5418
1,2022-10-05,BLOOMBERG,1.4015
2,2022-10-06,BLOOMBERG,1.6101
3,2022-10-07,BLOOMBERG,1.6264
4,2022-10-08,BLOOMBERG,1.1735


So now we have two dataframes: one for Ukraine and one for Russia. In either, there are three columns: date, station, and relative frequency of keyword mentions (from left to right).



### Clean Data for Further Use

In [34]:
# Rename the first column to something shorter for convenience
df_ukr = df_ukr.rename(columns={df_ukr.columns[0]: "date_col"})
df_rus = df_rus.rename(columns={df_rus.columns[0]: "date_col"})

In [35]:
# Transform the first column to the datetime format
df_ukr['date_col'] = pd.to_datetime(df_ukr['date_col'])
df_rus['date_col'] = pd.to_datetime(df_rus['date_col'])

In [40]:
# Select three stations for comparison
# CNN: Presumed to represent an ideological middle ground
# FOXNEWS: Presumed to represent the ideological conservative
# MSNBC: Presumed to represent the ideological liberal
df_rus = df_rus[[x in ['CNN', 'FOXNEWS', 'MSNBC'] for x in df_rus.Series]]
df_ukr = df_ukr[[x in ['CNN', 'FOXNEWS', 'MSNBC'] for x in df_ukr.Series]]

### Initiate a Dashboard Instance



In [46]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LITERA])
server = app.server

Code explanation: This is just the formalities of creating a dashboard. To use a template that controls how our dashboard will look, we use the LITERA theme from [Dash Bootstrap Components](https://dash-bootstrap-components.opensource.faculty.ai/)(`dbc`). You can choose any theme you prefer from [this list](https://dash-bootstrap-components.opensource.faculty.ai/docs/themes/).  

### Coding the Frontend

In [58]:
app.layout = dbc.Container(
    [   dbc.Row([ # row 1
        dbc.Col([html.H1('US National Television News Coverage of the War in Ukraine')],
        className="text-center mt-3 mb-1")
    ]
    ),
        dbc.Row([ # row 2
            dbc.Label("Select a date range:", className="fw-bold")
    ]),
    
     dbc.Row([ # row 3
              dcc.DatePickerRange(
                id='date-range',
                min_date_allowed=df_ukr['date_col'].min().date(),
                max_date_allowed=df_ukr['date_col'].max().date(),
                initial_visible_month=df_ukr['date_col'].min().date(),
                start_date=df_ukr['date_col'].min().date(),
                end_date=df_ukr['date_col'].max().date()
              )
    ]),

     dbc.Row([ # row 4
              dbc.Col(dcc.Graph(id='line-graph-ukr'), 
                      )
     ]),

    dbc.Row([ # row 5
              dbc.Col(dcc.Graph(id='line-graph-rus'), 
                      )
     ])

    ])

Code explanation: Here, we need to think about the dashboard layout as a grid with rows and columns. In our dashboard, we have five rows from top to bottom: title, instruction text for the date-range selector, data-range selector, the first line graph, and the second line graph. If you want to add columns within a row, you can easily do so by nesting two `dbc.Col` components under the same `dbc.Row` component. Below is an example of placing the two line graphs side by side on the same row:

In [56]:
dbc.Row([
          dbc.Col(dcc.Graph(id='line-graph-ukr'), 
                  ),
          dbc.Col(dcc.Graph(id='line-graph-rus'), 
                  )
  ])

Also important to note in the frontend code above is that we explicitly give names to those components that are involved in user interaction. In our case, we have three such components: the data-range selector as input and the two line graphs as output (i.e., reacting to any update in the date-range selector triggered by a user). The names of these components are created using the `id` parameter. These names are very important when we code the backend.

### Coding the Backend

In [48]:
# callback decorator
@app.callback(
    Output('line-graph-ukr', 'figure'),
    Output('line-graph-rus', 'figure'),
    Input('date-range', 'start_date'),
    Input('date-range', 'end_date')   
)

# callback function
def update_output(start_date, end_date):
    # filter dataframes based on updated data range
    mask_ukr = (df_ukr['date_col'] >= start_date) & (df_ukr['date_col'] <= end_date)
    mask_rus = (df_rus['date_col'] >= start_date) & (df_rus['date_col'] <= end_date)
    df_ukr_filtered = df_ukr.loc[mask_ukr]
    df_rus_filtered = df_rus.loc[mask_rus]
    
    # create line graphs based on filtered dataframes
    line_fig_ukr = px.line(df_ukr_filtered, x="date_col", y="Value", 
                     color='Series', title="Coverage of Ukranian Keywords")
    line_fig_rus = px.line(df_rus_filtered, x='date_col', y='Value', 
                     color='Series', title="Coverage of Russian Keywords")

    # set x-axis title and y-axis title in line graphs 
    line_fig_ukr.update_layout(
                   xaxis_title='Date',
                   yaxis_title='Percentage of Airtime')
    line_fig_rus.update_layout(
                   xaxis_title='Date',
                   yaxis_title='Percentage of Airtime')
    
    # set label format on y-axis in line graphs 
    line_fig_ukr.update_xaxes(tickformat="%b %d<br>%Y")
    line_fig_rus.update_xaxes(tickformat="%b %d<br>%Y")
    
    return line_fig_ukr, line_fig_rus

Code explanation: In the backend, the core concepts are *callback decorator* and *callback function*. In the above code, `@app.callback`, the callback decorator, defines which output variables and input variables are included in a user interaction. For example, remember that when we code the frontend, we name the line graph for Ukraine as 'line-graph-ukr'. Now we refer this name in one of our Output variable. The parameter 'figure' specifies which property of the referred component is updated when needed.<br>

The callback function, `update_output()`, defines how the interaction occurs: The two line graphs are updated whenever the start date or the end date in the date-range selector is changed by a user. This is called *reactive programming*, similar to [the server logic used in R Shiny](https://programminghistorian.org/en/lessons/shiny-leaflet-newspaper-map-tutorial#shiny-and-reactive-programming). More detailed explanations are provided as comments in the above code. Note that the two returned objects (`line_fig_ukr` and `line_fig_rus`) should be ordered in the same way as how the output variables are ordered in the callback decorator (i.e., Ukraine's line graph goes first).

### Testing the Dashboard

In [59]:
app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>

Code explanation: Now we can run the above code to actually see and test the created dashboard. It is recommended to turn on the debug mode so that any errors can be looked into when needed.

## Deploying the Dashboard (to be written)
it looks like heroku doesn't host free apps no more, my plan is to explore [render](https://render.com/docs/deploy-flask).com [suggested by the dash community](https://community.plotly.com/t/heroku-alternatives/67399/5). 

## Conclusion (to be written)


# ignore

In [43]:
# Show dashboard inside the notebook
app.run_server(debug=True, mode="inline")

TypeError: ignored

In [52]:
# Initiate a dashboard instance
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LITERA])
server = app.server

# Dashboard front end
app.layout = dbc.Container(
    [   dbc.Row([
        dbc.Col([html.H1('US National Television News Coverage of the War in Ukraine')],
        #html.Hr()],
        className="text-center mt-3 mb-1")
    ]
    ),
        dbc.Row([
            dbc.Label("Select a date range:", className="fw-bold")
    ]),
    
     dbc.Row([
              dcc.DatePickerRange(
                id='date-range',
                min_date_allowed=df_ukr['date_col'].min().date(),
                max_date_allowed=df_ukr['date_col'].max().date(),
                initial_visible_month=df_ukr['date_col'].min().date(),
                start_date=df_ukr['date_col'].min().date(),
                end_date=df_ukr['date_col'].max().date()
              )
    ]),

     dbc.Row([
              dbc.Col(dcc.Graph(id='line-graph-ukr'), 
                      )
     ]),

    dbc.Row([
              dbc.Col(dcc.Graph(id='line-graph-rus'), 
                      )
     ])

    ])

# Dashboard back end
@app.callback(
    Output('line-graph-ukr', 'figure'),
    Output('line-graph-rus', 'figure'),
    Input('date-range', 'start_date'),
    Input('date-range', 'end_date')   
)
def update_output(start_date, end_date):
    mask_ukr = (df_ukr['date_col'] >= start_date) & (df_ukr['date_col'] <= end_date)
    mask_rus = (df_rus['date_col'] >= start_date) & (df_rus['date_col'] <= end_date)

    df_ukr_filtered = df_ukr.loc[mask_ukr]
    df_rus_filtered = df_rus.loc[mask_rus]
    
    line_fig_ukr = px.line(df_ukr_filtered, x="date_col", y="Value", 
                     color='Series', title="Coverage of Ukranian Keywords")
    line_fig_rus = px.line(df_rus_filtered, x='date_col', y='Value', 
                     color='Series', title="Coverage of Russian Keywords")

    line_fig_ukr.update_layout(
                   xaxis_title='Date',
                   yaxis_title='Percentage of Airtime')
    
    line_fig_rus.update_layout(
                   xaxis_title='Date',
                   yaxis_title='Percentage of Airtime')

    line_fig_ukr.update_xaxes(tickformat="%b %d<br>%Y")
    line_fig_rus.update_xaxes(tickformat="%b %d<br>%Y")
    
    return line_fig_ukr, line_fig_rus

In [ ]:
requests.get(queryurl)

In [ ]:
query_cheney = f"https://api.gdeltproject.org/api/v2/tv/tv?query=(cheney%20%28context%3A%22nomination%22%20OR%20context%3A%22election%22%20OR%20context%3A%22presidential%22%29)%20market:%22National%22&mode=timelinevol&format=html&datanorm=perc&format=csv&timelinesmooth=5&datacomb=sep&timezoom=yes&STARTDATETIME=20220101120000&ENDDATETIME={today_str}120000"

In [ ]:
query_cheney = f"https://api.gdeltproject.org/api/v2/tv/tv?query=(cheney%20context:%22nomination%22%20OR%20context:%22election%22%20OR%20context:%22presidential%22)%20market:%22National%22&mode=timelinevol&format=html&datanorm=perc&format=csv&timelinesmooth=5&datacomb=sep&timezoom=yes&STARTDATETIME=20220101120000&ENDDATETIME={today_str}120000"

In [ ]:
query_cheney = f"https://api.gdeltproject.org/api/v2/tv/tv?query=cheney%20(context:%22nomination%22%20OR%20context:%22election%22%20OR%20context:%22presidential%22)%20market:%22National%22&mode=timelinevol&format=html&datanorm=perc&format=csv&datacomb=sep&STARTDATETIME=20220101120000&ENDDATETIME={today_str}120000"

In [ ]:
query_cheney_cnn = "https://api.gdeltproject.org/api/v2/tv/tv?query=cheney%20(context:%22election%22%%20OR%20context:%22election%22%20OR%20context:%22presidential%22)%20network:CNN&mode=timelinevol&format=html&datanorm=perc&format=csv&datacomb=sep&STARTDATETIME=20220701120000&ENDDATETIME=20220710120000"

In [ ]:
df_cheney = to_df(query_cheney)

In [ ]:
df_cheney_cnn = to_df(query_cheney_cnn)

In [ ]:
df_cheney_cnn

,Date (Daily +00:00: 07/01/2022 - 07/10/2022),Series,Value
0,2022-07-01,CNN,0.5039
1,2022-07-02,CNN,0.3875
2,2022-07-03,CNN,0.2087
3,2022-07-04,CNN,0.1099
4,2022-07-05,CNN,0.0458
5,2022-07-06,CNN,0.0458
6,2022-07-07,CNN,0.0461
7,2022-07-08,CNN,0.0452
8,2022-07-09,CNN,0.0000
9,2022-07-10,CNN,0.0681


In [ ]:
df_cheney

,Date (Daily +00:00: 01/01/2022 - 07/10/2022),Series,Value
0,2022-01-01,BLOOMBERG,0.0000
1,2022-01-02,BLOOMBERG,0.0000
2,2022-01-03,BLOOMBERG,0.0000
3,2022-01-04,BLOOMBERG,0.0000
4,2022-01-05,BLOOMBERG,0.0000
...,...,...,...
1714,2022-07-06,MSNBC,0.0218
1715,2022-07-07,MSNBC,0.0901
1716,2022-07-08,MSNBC,0.0451
1717,2022-07-09,MSNBC,0.0000


In [ ]:
requests.get(query_cheney)

<Response [200]>

In [ ]:
df_cheney[[x in ['CNN'] for x in df_cheney.Series]]

,Date (Daily +00:00: 01/01/2022 - 07/10/2022),Series,Value
374,2022-01-05,CNN,6.0921
375,2022-01-06,CNN,9.3895
376,2022-01-07,CNN,10.6201
377,2022-01-08,CNN,10.6026
378,2022-01-09,CNN,9.5580
...,...,...,...
556,2022-07-06,CNN,5.9535
557,2022-07-07,CNN,5.2038
558,2022-07-08,CNN,5.4129
559,2022-07-09,CNN,5.7679


In [ ]:
df_cheney[[x in ['CNN'] for x in df_cheney.Series]]

,Date (Daily +00:00: 01/01/2022 - 07/10/2022),Series,Value
382,2022-01-01,CNN,0.0222
383,2022-01-02,CNN,0.1125
384,2022-01-03,CNN,0.1614
385,2022-01-04,CNN,0.1314
386,2022-01-05,CNN,0.0000
...,...,...,...
568,2022-07-06,CNN,0.0458
569,2022-07-07,CNN,0.0461
570,2022-07-08,CNN,0.0452
571,2022-07-09,CNN,0.0000


# Recycle bin:

In [ ]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LITERA])
server = app.server

In [ ]:
from datetime import date

In [ ]:
mask_ukr = (df_ukr['date_col'].dt.date >= date(2022, 3, 1)) & (df_ukr['date_col'].dt.date <= date(2022, 3, 31))
mask_rus = (df_rus['date_col'].dt.date >= date(2022, 3, 1)) & (df_rus['date_col'].dt.date <= date(2022, 3, 31))
df_ukr_filtered = df_ukr.loc[mask_ukr]
df_rus_filtered = df_rus.loc[mask_rus]

print(df_ukr_filtered['date_col'])

247    2022-03-01
248    2022-03-02
249    2022-03-03
250    2022-03-04
251    2022-03-05
          ...    
1011   2022-03-27
1012   2022-03-28
1013   2022-03-29
1014   2022-03-30
1015   2022-03-31
Name: date_col, Length: 93, dtype: datetime64[ns]


In [ ]:
fig = px.line(df_ukr_filtered, x="date_col", y="Value", 
                     color='Series', title="UKR Results")

In [ ]:
fig.update_xaxes(
    #dtick="M1",
    tickformat="%b %d<br>%Y")
fig.show()

In [ ]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LITERA])
server = app.server

app.layout = dbc.Container(
    [   dbc.Row([
        dbc.Col([html.H1('News Coverage TV Keyword')],
        #html.Hr()],
        className="text-center mt-3 mb-1")
    ]
    ),
        dbc.Row([

        dbc.Col([
            dbc.Label("Drag to modify date range:", className="fw-bold"),
            dcc.DatePickerRange(
                id='date-range',
                min_date_allowed=df_ukr['date_col'].min().date(),
                max_date_allowed=df_ukr['date_col'].max().date(),
                initial_visible_month=df_ukr['date_col'].min().date(),
                start_date=df_ukr['date_col'].min().date(),
                end_date=df_ukr['date_col'].max().date()
              )
             ],
        #width={'size': 8, "offset": 1}
        )
    ]),
    #  dbc.Row([
    #           dbc.Col(dcc.Graph(id='graph-ukr'), 
    #                   width={'size': 6, 'offset': 0}),
    #           dbc.Col(dcc.Graph(id='graph-rus'), 
    #                   width={'size': 6, 'offset': 0})
    #  ]),
    #  dbc.Row([
    #           dbc.Col(dcc.Graph(id='line-graph-ukr'), 
    #                   width={'size': 6, 'offset': 0}),
    #           dbc.Col(dcc.Graph(id='line-graph-rus'), 
    #                   width={'size': 6, 'offset': 0})
    #  ])
     dbc.Row([
              dbc.Col(dcc.Graph(id='line-graph-ukr'), 
                      )
     ]),

    dbc.Row([
              dbc.Col(dcc.Graph(id='line-graph-rus'), 
                      )
     ])

    ])

@app.callback(
    # Output('graph-ukr', 'figure'),
    # Output('graph-rus', 'figure'),
    Output('line-graph-ukr', 'figure'),
    Output('line-graph-rus', 'figure'),
    Input('date-range', 'start_date'),
    Input('date-range', 'end_date')   
)
def update_output(start_date, end_date):
    mask_ukr = (df_ukr['date_col'] >= start_date) & (df_ukr['date_col'] <= end_date)
    mask_rus = (df_rus['date_col'] >= start_date) & (df_rus['date_col'] <= end_date)

    df_ukr_filtered = df_ukr.loc[mask_ukr]
    df_rus_filtered = df_rus.loc[mask_rus]

    print(df_ukr_filtered['Series'])

    #fig_ukr = px.bar(df_ukr_filtered, x="Series", y="Value", title="UKR Results")
    #fig_rus = px.bar(df_rus_filtered, x="Series", y="Value", title="RUS Results")

    # fig_ukr = px.bar(df_ukr_filtered.groupby(['Series']).mean().reset_index(), 
    #                  x="Series", y="Value", title="UKR Results")
    # fig_rus = px.bar(df_rus_filtered.groupby(['Series']).mean().reset_index(), 
    #                  x="Series", y="Value", title="RUS Results")
    
    line_fig_ukr = px.line(df_ukr_filtered, x="date_col", y="Value", 
                     color='Series', title="Coverage of Ukranian Keywords")
    line_fig_rus = px.line(df_rus_filtered, x='date_col', y='Value', 
                     color='Series', title="Coverage of Russian Keywords")

    # line_fig_ukr.update_layout(
    #     xaxis=dict(
    #         # showline=True,
    #         # showgrid=False,
    #         # showticklabels=True,
    #         # linecolor='rgb(204, 204, 204)',
    #         # linewidth=2,
    #         # ticks='outside',
    #         tickfont=dict(
    #             #family='Arial',
    #             size=8,
    #             #color='rgb(82, 82, 82)',
    #         ),
    #     )
    # )

    line_fig_ukr.update_layout(
                   xaxis_title='Date',
                   yaxis_title='Percentage of Airtime')
    
    line_fig_rus.update_layout(
                   xaxis_title='Date',
                   yaxis_title='Percentage of Airtime')

    line_fig_ukr.update_xaxes(tickformat="%b %d<br>%Y")
    line_fig_rus.update_xaxes(tickformat="%b %d<br>%Y")
    #return fig_ukr, fig_rus, line_fig_ukr, line_fig_rus
    return line_fig_ukr, line_fig_rus

In [ ]:
app.run_server(debug=True, mode="inline")